In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from datasets import load_dataset, load_dataset_builder

In [54]:
import sys
sys.path.append('../..')

from baselines.galaxy10_decals.dataset_wrapper import DatasetWrapper

In [55]:
dataset_loc = 'mwalmsley/galaxy10_decals'

In [56]:
# Load Dataset Builders 
decals10_builder = load_dataset_builder(dataset_loc, 'astropile', trust_remote_code=True)
# decals10_builder.set_format('torch')

In [57]:
type(decals10_builder)

datasets_modules.datasets.mwalmsley--galaxy10_decals.f7f0215e9b20abfd8f74854d9ac987a9e7ef308f4a8681fa7840260fba92924c.galaxy10_decals.GZDecals

In [58]:
decals10_train = load_dataset('mwalmsley/galaxy10_decals', 'astropile', split='train', trust_remote_code=True)
decals10_test = load_dataset('mwalmsley/galaxy10_decals', 'astropile', split='test', trust_remote_code=True)

galaxy10_decals
galaxy10_decals


In [59]:

decals10_train.set_format('torch')
decals10_test.set_format('torch')

In [60]:
len(decals10_train)

200

In [61]:
decals10_train[0]['label']

tensor(5)

In [62]:
decals10_train[0]['images_by_band']['band']

['g', 'r', 'z']

In [63]:
decals10_train[0]['images_by_band']['array'].shape

torch.Size([3, 224, 224])

In [64]:
type(decals10_train)

datasets.arrow_dataset.Dataset

In [65]:
wrapper = DatasetWrapper(
    train_dataset=decals10_train,
    test_dataset=decals10_test,
    feature_flag='images_by_band.array',
    label_flag='label',
    loading='iterated'
)

In [66]:
# decals10_train['images_by_band']

In [67]:
wrapper.prepare_data()

Computing statistics for images_by_band.array: 100%|██████████| 2/2 [00:15<00:00,  7.79s/it]


In [68]:
wrapper.feature_mean, wrapper.feature_std

(tensor([[[0.0065]],
 
         [[0.0129]],
 
         [[0.0205]]]),
 tensor([[[0.0664]],
 
         [[0.1309]],
 
         [[0.1783]]]))

In [69]:
train_loader = wrapper.train_dataloader()
val_loader = wrapper.val_dataloader()
test_loader = wrapper.test_dataloader()

In [70]:
for images, labels in train_loader:
    print(images.shape, labels.shape)

torch.Size([128, 3, 224, 224]) torch.Size([128])
torch.Size([32, 3, 224, 224]) torch.Size([32])


In [71]:
im_np = images[0].numpy().transpose(1, 2, 0)
im_np.min(), im_np.max()

(-0.11532954, 2.002871)

In [ ]:
# alternative non-lightning loading option

In [72]:
# target_size = 64
# from torchvision.transforms import Compose, Normalize, Resize, Lambda, RandomHorizontalFlip, RandomVerticalFlip


# normalize = Normalize(mean=1., std=1.)
#                     #   https://pytorch.org/vision/main/generated/torchvision.transforms.Lambda.html)

# _transforms = Compose(
#     [
#         Resize(target_size),
#         RandomHorizontalFlip(),
#         RandomVerticalFlip(),
#         Lambda(func=lambda x: np.arcsinh(x)),
#         normalize
#     ]
# )

# def transforms(examples):
#     examples["pixel_values"] = [_transforms(image.convert("RGB")) for image in examples["image"]]
#     return examples